### ML Model for Stock Return Prediction
Using Richard's ceo_gender_file
Incorporates text info with bag of words embedding

In [1]:
import pandas as pd
import numpy as np
from numpy import loadtxt
import keras
from keras.models import Sequential
from keras.layers import Dense
import os
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import nltk
import string
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def cleantxt(txt):
    """
    Cleans the string passed. Cleaning Includes-
    1. remove special characters/symbols
    2. convert text to lower-case
    3. retain only alphabets
    4. remove words less than 3 characters
    5. remove stop-words
    """  
    # collecting english stop words from nltk-library
    stpw = stopwords.words('english')
    
    # Adding custom stop-words
    stpw.extend(['www','http','utc'])
    stpw = set(stpw)
    
    # using regex to clean the text
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-z ]", " ", txt)
    txt = re.sub(r"\b\w{1,3}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    return txt


In [3]:
# create an array with (CIK: strategy text)
corpus_dict = dict()
corpus = [] # list of strategy text
ncols = 0
with open("ceo_gender_training.csv") as f:
    ncols = len(f.readline().split(','))
ceo_data = pd.read_csv("ceo_gender_training.csv")
ceo_data.rename(columns={ceo_data.columns[0]: "pid" }, inplace = True)
ceo_data['has_file'] = 0
company_doc = []
for index, row in ceo_data.iterrows():
    folder_path = row['folder_path']
    QTR = folder_path.split("\\")[0]
    CIK = folder_path.split("\\")[1]
    try:
        f1 = open(QTR + "/" + CIK + "/" + "business-section.txt", "r")
        business = f1.read()
        f2 = open(QTR + "/" + CIK + "/" + "risk-factors-section.txt", "r")
        risk = f2.read()
        strategy = business + risk
        ceo_data.at[index,'has_file'] = 1
        clean_strategy = cleantxt(strategy)
        corpus_dict[CIK] = clean_strategy
        corpus.append(clean_strategy)
    except FileNotFoundError:
        ceo_data.at[index,'has_file'] = 0

ceo_data['has_file'].unique()
# consider only rows with the file data
ceo_data2 = ceo_data.loc[ceo_data['has_file'] == 1].reset_index()
print(ceo_data2.shape)

(1030, 39)


In [4]:
# creates vector embedding for the text using bag of words
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(corpus)
vectors = vectors.todense()
print(vectors.shape)

(1030, 60443)


In [5]:
# creates vector embedding for the text using TF-IDF
tfidf  = TfidfVectorizer()
vectors = tfidf.fit_transform(corpus)
vectors = vectors.todense()
print(vectors.shape)

(1030, 60443)


In [103]:
from sklearn.preprocessing import Binarizer
# creates vector embedding using one-hot encoding
freq   = CountVectorizer()
corpus2 = freq.fit_transform(corpus)

onehot = Binarizer()
vectors = onehot.fit_transform(corpus2.toarray())
print(vectors.shape)

(1030, 60443)


In [86]:
print(vectors)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [6]:
ceo_data_small = ceo_data2[["ret_0220_0325", "at", "lt", "che", "ni", "sale", "execrank", "exchange_id", 
                           "gender_id", "state_id", "days_becameceo", "zip", "FF_48", "shrout_0220"]]
pd_vectors = pd.DataFrame(vectors)
X_combined = pd.concat([ceo_data_small, pd_vectors], axis=1)
X_combined = X_combined.dropna()
X = X_combined.iloc[:,1:]
y = X_combined.iloc[:,0]
# split trian and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train.shape)

(777, 60456)


In [7]:
# define the keras model for deep learning
model = Sequential()
model.add(Dense(5, input_dim=60456, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_squared_error'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=10)

# evaluate the keras model
# _, mse = model.evaluate(X_test, y_test)
# print('MSE: %.2f' % (mse*100))
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test mse:", results)


Epoch 1/150
78/78 [==============================] - 1s 3ms/step - loss: 0.1409 - mean_squared_error: 0.1409
Epoch 2/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1412 - mean_squared_error: 0.1412
Epoch 3/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1460 - mean_squared_error: 0.1460
Epoch 4/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1420 - mean_squared_error: 0.1420
Epoch 5/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1422 - mean_squared_error: 0.1422
Epoch 6/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1479 - mean_squared_error: 0.1479
Epoch 7/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1455 - mean_squared_error: 0.1455
Epoch 8/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1384 - mean_squared_error: 0.1384
Epoch 9/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1379 - mean_squared_error: 0.1379
Epoch 10/150
78/78 

78/78 [==============================] - 0s 3ms/step - loss: 0.1346 - mean_squared_error: 0.1346
Epoch 76/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1445 - mean_squared_error: 0.1445
Epoch 77/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1460 - mean_squared_error: 0.1460
Epoch 78/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1389 - mean_squared_error: 0.1389
Epoch 79/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1446 - mean_squared_error: 0.1446
Epoch 80/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1389 - mean_squared_error: 0.1389
Epoch 81/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1455 - mean_squared_error: 0.1455
Epoch 82/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1363 - mean_squared_error: 0.1363
Epoch 83/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1377 - mean_squared_error: 0.1377
Epoch 84/150
78/78 [===

78/78 [==============================] - 0s 3ms/step - loss: 0.1382 - mean_squared_error: 0.1382
Epoch 149/150
78/78 [==============================] - 0s 3ms/step - loss: 0.1450 - mean_squared_error: 0.1450
Epoch 150/150
2/2 [==============================] - 0s 19ms/step - loss: 0.1612 - mean_squared_error: 0.1612
test loss, test mse: [0.16117793321609497, 0.16117793321609497]


In [8]:
# run the data on xgboost model
data_dmatrix = xgb.DMatrix(data=X,label=y)

# train xgb classifier
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

mse = mean_squared_error(y_test, preds)
print("MSE: %f" % (mse))


MSE: 0.021936


In [61]:
# considering applying dimensionality reduction
# scaler = StandardScaler()
# X_features = scaler.fit_transform(vectors2)

# pca = PCA()
# pca.fit_transform(X_features)
# pca_variance = pca.explained_variance_
# print(pca.explained_variance_ratio_)

[1.29603848e-02 7.68492611e-03 6.73325957e-03 ... 2.60343241e-33
 2.17261859e-34 9.25565100e-37]


In [9]:
from sklearn import svm
regr = svm.SVR()
regr.fit(X_train, y_train)
preds = regr.predict(X_test)
mse = mean_squared_error(y_test, preds)
print("MSE: %f" % (mse))

MSE: 0.025889


In [11]:
# Ridge Regression
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
mse = mean_squared_error(y_test, preds)
print("MSE: %f" % (mse))

MSE: 0.016896
